Explain how you can implement ML in a real world application.

Train an SVM regressor on : Bengaluru housing dataset

                  Must include in details:

                           - EDA

                            - Feature engineering 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

In [2]:
train = pd.read_csv(r"C:\Users\suzan\Desktop\iNeuron\test\Bengaluru_House_Data.csv")
train.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [3]:
train.shape

(13320, 9)

In [4]:
train.isnull().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

In [5]:
train.dtypes

area_type        object
availability     object
location         object
size             object
society          object
total_sqft       object
bath            float64
balcony         float64
price           float64
dtype: object

In [6]:
#applying label encoder on society variable
le = LabelEncoder()
train['society'] = le.fit_transform(train['society'])

In [7]:
train.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,464,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,2439,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,2688,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,2186,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,2688,1200,2.0,1.0,51.00


In [9]:
#using ordinal encoding on area_type
inputval = train['area_type'].values.reshape(-1,1)

In [10]:
oe = OrdinalEncoder(categories=[['Super built-up  Area','Built-up  Area','Plot  Area','Carpet  Area']])
train['labelled_areatype'] = oe.fit_transform(inputval)
train.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price,labelled_areatype
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,464,1056,2.0,1.0,39.07,0.0
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,2439,2600,5.0,3.0,120.00,2.0
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,2688,1440,2.0,3.0,62.00,1.0
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,2186,1521,3.0,1.0,95.00,0.0
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,2688,1200,2.0,1.0,51.00,0.0


In [11]:
train['labelled_areatype'].value_counts()

0.0    8790
1.0    2418
2.0    2025
3.0      87
Name: labelled_areatype, dtype: int64

In [12]:
# splits the range values and return their mean value , and fill None for other string formats
def preprocess_sqft(value):
    if '-' in value:
        start,end = value.split('-')
        start = float(start.strip())
        end = float(end.strip())
        return (start+end)/2
    else :
        try:
            return float(value)
        except ValueError:
            return None

In [13]:
train['total_sqft']=train['total_sqft'].apply(lambda x: preprocess_sqft(x))

In [14]:
train['total_sqft'].head()

0    1056.0
1    2600.0
2    1440.0
3    1521.0
4    1200.0
Name: total_sqft, dtype: float64

In [15]:
#extract the numerical values
train['bedrooms'] = train['size'].str.extract('(\d+)').astype(float)

In [16]:
train.isnull().sum()

area_type              0
availability           0
location               1
size                  16
society                0
total_sqft            46
bath                  73
balcony              609
price                  0
labelled_areatype      0
bedrooms              16
dtype: int64

In [18]:
#imputing missing values
train['bedrooms'] = train['bedrooms'].fillna(train['bedrooms'].mean())
train['balcony'] = train['balcony'].fillna(train['balcony'].mean())
train['total_sqft'] = train['total_sqft'].fillna(train['total_sqft'].mean())
train['bath'] = train['bath'].fillna(train['bath'].mean())

In [19]:
train.isnull().sum()

area_type             0
availability          0
location              1
size                 16
society               0
total_sqft            0
bath                  0
balcony               0
price                 0
labelled_areatype     0
bedrooms              0
dtype: int64

In [20]:
xtrain = train.drop(columns=['area_type','availability','location','size','price'])
ytrain = train['price']

In [21]:
xtrain.isnull().sum()

society              0
total_sqft           0
bath                 0
balcony              0
labelled_areatype    0
bedrooms             0
dtype: int64

In [22]:
x_train,x_test,y_train,y_test = train_test_split(xtrain,ytrain,test_size=0.25,random_state=42)

In [23]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((9990, 6), (3330, 6), (9990,), (3330,))

In [ ]:
model1 = SVR(kernel='poly', degree=5)
model1.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Assuming you have x_test and y_test
predictions = model1.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r2)
